In [8]:
using Luxor
using LinearAlgebra: norm

In [9]:
mutable struct Instance
    n :: Int64
    D
    l
#     c
#     function Instance(n)
#         D = zeros(n+1,n+1)
#         c = rand(2,n)
#         for i = 1:n, j = 1:n
#             D[i,j]=norm(c[:,i]-c[:,j])
#         end
#         D[n+1,:]=D[1,:]
#         D[:,n+1]=D[:,1]
#         new(n,D,c)
#     end
    function Instance(s)
        D = zeros(1,1)
#         c = rand(2,n) # not implemented
        nNodes = 0
        l = 0
        open(s) do f
            line = 1
            lineContent = readline(f)
            lineSplited = split(lineContent, " ")
            nNodes = parse(Int64, lineSplited[1])
            l = parse(Int64, lineSplited[2])
            D = zeros(nNodes,nNodes)
            while ! eof(f)
                lineContent = readline(f)
                lineSplited = split(lineContent, " ")
                for col = 1:nNodes
                    D[line,col] = parse(Float64, lineSplited[col])
                end
                
                line += 1
            end
        end
        new(nNodes,D,l) #,c
    end
end
# point(I,i) = Point(I.c[1,i],I.c[2,i])

In [10]:
I = Instance("instances/rml1.txt")

Instance(442, [0.0 100.0 … 557.0 447.0; 100.0 0.0 … 550.0 539.0; … ; 557.0 550.0 … 0.0 896.0; 447.0 539.0 … 896.0 0.0], 50)

In [11]:
mutable struct Solution
    π :: Array{Int64,1} ## solução: uma permutação de cidades
    v :: Float64        ## valor da solução: distância total
    Solution(s::Solution) = new(copy(s.π),s.v)
    Solution(π,v) = new(copy(π),v)
end
Base.copy(s::Solution) = Solution(s)

In [16]:
function vmp(I::Instance)
    s = Solution([1],0.0)
    free=trues(I.n)
    free[1]=false
    for k=2:I.n
      f=findall(free)
      j=f[findmin(I.D[s.π[end],f])[2]]
    #       s.v += I.D[s.π[end],j]
      push!(s.π,j)
      free[j]=false
    end
    for lookahead=1:I.l
        for i=1:I.n
#             println("$(s.π[i]) - $(s.π[((i+lookahead-1)%(I.n))+1])")
#             println("$(I.D[i,((i+lookahead-1)%(I.n))+1])")
            s.v += I.D[s.π[i],s.π[((i+lookahead-1)%(I.n))+1]]
        end
    end
    push!(s.π,1)
    s
end

vmp (generic function with 1 method)

In [17]:
s=vmp(I)
# println(s.π)
println("Vizinho mais próximo: $(s.v).")

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 376, 377, 33, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 378, 88, 89, 90, 91, 92, 93, 94, 95, 379, 96, 380, 97, 98, 383, 384, 113, 125, 135, 148, 160, 171, 184, 199, 210, 220, 226, 411, 410, 414, 237, 265, 420, 268, 416, 264, 263, 236, 262, 261, 260, 259, 258, 257, 256, 255, 254, 253, 252, 251, 250, 415, 249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 235, 228, 406, 401, 186, 173, 162, 150, 137, 127, 386, 115, 104, 441, 102, 103, 114, 126, 136, 149, 161, 172, 185, 400, 405, 227, 234, 238, 239, 266, 269, 273, 276, 279, 281, 282, 428, 342, 341, 346, 347, 348, 433, 349, 350, 351, 352, 343, 353, 354, 355, 434, 356, 357, 358, 435, 359, 360, 361, 344, 362, 363, 364, 365, 366, 367, 345, 368, 369, 370, 3

In [18]:
function bl_2ex(I::Instance,s₀::Solution)
    s=copy(s₀)
    move=(1,1)
    v=0.0
    while move!=(0,0)
        move=(0,0)
        for a=1:I.n, b=(a+2):I.n
            a==1 && b==I.n && continue
            
#             newπ = s.π 
#             aux = newπ[a+1] #swap
#             newπ[a+1] = newπ[b]
#             newπ[b] = aux

#             v = 0
#             for lookahead=1:I.l
#                 for i=1:I.n
#                     v += I.D[newπ[i],newπ[((i+lookahead-1)%(I.n))+1]]
#                 end
#             end
            
#             v=s.v-I.D[s.π[a],s.π[a+1]]-I.D[s.π[b],s.π[b+1]]+I.D[s.π[a],s.π[b]]+I.D[s.π[a+1],s.π[b+1]]
            v=s.v - I.l * (I.D[s.π[a],s.π[a+1]] + I.D[s.π[b],s.π[b+1]]) + I.l * (I.D[s.π[a],s.π[b]] + I.D[s.π[a+1],s.π[b+1]])
            
            if v<s.v
                move=(a,b)
                break ## first improvement
            end
        end
        if move!=(0,0)
            ## reverse segment
            move=(move[1]+1,move[2])
            while move[1]<move[2]
                s.π[move[1]],s.π[move[2]] = s.π[move[2]],s.π[move[1]]
                move=(move[1]+1,move[2]-1)
            end
            s.v=v
        end
    end
    s
end

bl_2ex (generic function with 1 method)

In [20]:
# s′′=bl_2ex(I,s)
# println(s′′)
# println("Depois da busca local 2-exchange: $(s′′.v).")

Solution([1, 35, 2, 3, 4, 5, 6, 40, 41, 7, 8, 9, 10, 42, 43, 44, 11, 12, 45, 78, 79, 80, 100, 130, 142, 155, 166, 178, 398, 179, 395, 394, 192, 204, 216, 225, 233, 408, 409, 404, 217, 205, 194, 181, 193, 180, 167, 156, 143, 390, 388, 119, 108, 439, 84, 83, 82, 81, 50, 49, 48, 47, 46, 13, 14, 15, 16, 17, 18, 51, 19, 20, 21, 22, 23, 56, 57, 24, 25, 26, 58, 59, 27, 28, 29, 30, 31, 32, 376, 377, 33, 65, 64, 63, 62, 61, 60, 92, 91, 90, 89, 88, 378, 87, 55, 54, 53, 52, 85, 86, 382, 381, 109, 120, 131, 144, 391, 121, 385, 110, 122, 132, 145, 157, 168, 111, 123, 133, 146, 158, 169, 182, 197, 196, 195, 206, 207, 208, 218, 219, 209, 198, 183, 170, 159, 147, 134, 124, 112, 436, 101, 93, 94, 95, 379, 96, 380, 97, 98, 384, 383, 113, 125, 135, 148, 160, 171, 184, 199, 210, 220, 226, 237, 265, 414, 411, 410, 264, 416, 268, 420, 272, 423, 438, 422, 419, 262, 263, 236, 413, 412, 261, 260, 259, 258, 257, 256, 255, 254, 232, 224, 215, 203, 191, 190, 189, 201, 202, 402, 214, 223, 231, 252, 253, 418, 417, 